In [ ]:
import random
import os
import sys

# Check if running in Google Colab
try:
    # This will only work in Colab
    from google.colab import output
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Install and import ipywidgets properly for Colab
    try:
        import ipywidgets as widgets
        from IPython.display import display, HTML, clear_output
    except ImportError:
        import sys
        !{sys.executable} -m pip install ipywidgets
        import ipywidgets as widgets
        from IPython.display import display, HTML, clear_output
else:
    # Regular tkinter import for VS Code
    import tkinter as tk


class ProbabilityGames:
    def __init__(self):
        self.current_game = "Coin Toss Guessing Game"
        self.deck = []
        self.current_card = None

    # --- Coin Toss Guessing Game ---
    def play_coin_toss(self, guess):
        guess = guess.strip().capitalize()
        outcome = random.choice(["Heads", "Tails"])
        if guess == outcome:
            result_text = f"The coin landed on {outcome}. You guessed it!"
        elif guess not in ["Heads", "Tails"]:
            result_text = "Invalid guess. Try again."
        else:
            result_text = f"The coin landed on {outcome}. You were wrong!"
        return result_text

    # --- Card Rank Higher/Lower Game ---
    def create_deck(self):
        suits = ["Hearts", "Diamonds", "Clubs", "Spades"]
        ranks_order = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"]
        return [(suit, rank) for suit in suits for rank in ranks_order]

    def get_card_value(self, card):
        rank = card[1]
        ranks_order = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"]
        return ranks_order.index(rank)

    def draw_new_card(self):
        self.deck = self.create_deck()
        self.current_card = random.choice(self.deck)
        self.deck.remove(self.current_card)
        suit, rank = self.current_card
        card_text = f"Drawn card: {rank} of {suit}"
        return card_text

    def guess_card(self, higher=True):
        if not self.current_card:
            return "No current card. Draw a card first."
        if not self.deck:
            return "No more cards in the deck!"

        next_card = random.choice(self.deck)
        self.deck.remove(next_card)
        current_value = self.get_card_value(self.current_card)
        next_value = self.get_card_value(next_card)
        next_suit, next_rank = next_card
        next_card_text = f"Next card: {next_rank} of {next_suit}"

        if higher:
            result = "You guessed Higher correctly!" if next_value > current_value else "Oops! The next card was not higher."
        else:
            result = "You guessed Lower correctly!" if next_value < current_value else "Oops! The next card was not lower."

        self.current_card = None
        return f"{next_card_text}\n{result}"

    # --- Dice Roll Target Sum Game ---
    def play_dice_roll(self, num_dice, target_sum):
        try:
            num_dice = int(num_dice)
            target_sum = int(target_sum)
            if num_dice <= 0:
                return "Please enter a valid number of dice."
            if target_sum < num_dice or target_sum > 6 * num_dice:
                return f"Target sum should be between {num_dice} and {6 * num_dice}."
        except ValueError:
            return "Invalid input for number of dice or target sum."

        rolls = [random.randint(1, 6) for _ in range(num_dice)]
        actual_sum = sum(rolls)
        dice_result_text = f"You rolled: {', '.join(map(str, rolls))} (Sum: {actual_sum})"

        if actual_sum == target_sum:
            result_text = f"{dice_result_text}\nCongratulations! You hit the target sum ({target_sum})!"
        else:
            result_text = f"{dice_result_text}\nOops! You rolled {actual_sum}, but the target was {target_sum}."

        return result_text


# Implementation for VS Code (tkinter)
class ProbabilityGamesGUI:
    def __init__(self, master):
        self.master = master
        master.title("Interesting Probability Games")
        self.games = ProbabilityGames()

        self.current_game = tk.StringVar(master)
        self.current_game.set("Coin Toss Guessing Game")
        self.current_game.trace_add("write", self.on_game_change)

        game_options = ["Coin Toss Guessing Game", "Card Rank Higher/Lower", "Dice Roll Target Sum"]
        self.game_menu_label = tk.Label(master, text="Select Game:")
        self.game_menu_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")
        self.game_menu = tk.OptionMenu(master, self.current_game, *game_options)
        self.game_menu.grid(row=0, column=1, padx=10, pady=10, sticky="ew")

        self.welcome_label = tk.Label(master, text="Welcome to Interesting Probability Games!", font=("Arial", 14))
        self.welcome_label.grid(row=1, column=0, columnspan=2, pady=10)

        self.game_frame = tk.Frame(master)
        self.game_frame.grid(row=2, column=0, columnspan=2, padx=20, pady=20)

        self.displayed_widgets = {}

        # Show the initial game
        self.show_game_widgets(self.current_game.get())

    def on_game_change(self, *args):
        selected_game = self.current_game.get()
        # Reset welcome label when changing games
        self.welcome_label.config(text=f"Welcome to {selected_game}!")
        self.show_game_widgets(selected_game)

    def show_game_widgets(self, game):
        # Remove previous game widgets
        for widget in self.displayed_widgets.values():
            widget.grid_forget()
        self.displayed_widgets = {}

        if game == "Coin Toss Guessing Game":
            self.displayed_widgets["coin_guess_label"] = tk.Label(self.game_frame, text="Guess (Heads/Tails):")
            self.displayed_widgets["coin_guess_entry"] = tk.Entry(self.game_frame)
            self.displayed_widgets["flip_button"] = tk.Button(self.game_frame, text="Flip Coin", command=self.play_coin_toss)
            self.displayed_widgets["coin_outcome_label"] = tk.Label(self.game_frame, text="", font=("Arial", 18))
        elif game == "Card Rank Higher/Lower":
            self.displayed_widgets["card_drawn_label"] = tk.Label(self.game_frame, text="Click 'Draw' to start.", font=("Arial", 16))
            self.displayed_widgets["higher_button"] = tk.Button(self.game_frame, text="Higher", command=self.guess_higher, state=tk.DISABLED)
            self.displayed_widgets["lower_button"] = tk.Button(self.game_frame, text="Lower", command=self.guess_lower, state=tk.DISABLED)
            self.displayed_widgets["draw_button"] = tk.Button(self.game_frame, text="Draw Card", command=self.draw_new_card)
            self.displayed_widgets["card_result_label"] = tk.Label(self.game_frame, text="", font=("Arial", 18))
        elif game == "Dice Roll Target Sum":
            self.displayed_widgets["num_dice_label"] = tk.Label(self.game_frame, text="Number of Dice:")
            self.displayed_widgets["num_dice_entry"] = tk.Entry(self.game_frame)
            self.displayed_widgets["num_dice_entry"].insert(0, "2")
            self.displayed_widgets["target_sum_label"] = tk.Label(self.game_frame, text="Target Sum:")
            self.displayed_widgets["target_sum_entry"] = tk.Entry(self.game_frame)
            self.displayed_widgets["roll_dice_button"] = tk.Button(self.game_frame, text="Roll Dice", command=self.play_dice_roll)
            self.displayed_widgets["dice_result_label"] = tk.Label(self.game_frame, text="", font=("Arial", 18))

        # Display the widgets for the selected game
        row_num = 0
        for widget in self.displayed_widgets.values():
            widget.grid(row=row_num, column=0, columnspan=2, padx=5, pady=5, sticky="ew")
            row_num += 1

    # --- Coin Toss Guessing Game ---
    def play_coin_toss(self):
        guess = self.displayed_widgets["coin_guess_entry"].get()
        result_text = self.games.play_coin_toss(guess)
        self.displayed_widgets["coin_outcome_label"].config(text=result_text)
        self.welcome_label.config(text=result_text)

    # --- Card Rank Higher/Lower Game ---
    def draw_new_card(self):
        card_text = self.games.draw_new_card()
        self.displayed_widgets["card_drawn_label"].config(text=card_text)
        self.displayed_widgets["higher_button"].config(state=tk.NORMAL)
        self.displayed_widgets["lower_button"].config(state=tk.NORMAL)
        self.displayed_widgets["card_result_label"].config(text="")
        self.welcome_label.config(text=f"Card drawn! Will the next card be higher or lower?")

    def guess_higher(self):
        result = self.games.guess_card(higher=True)
        self.displayed_widgets["card_result_label"].config(text=result)
        result_parts = result.split("\n")
        if len(result_parts) > 1:
            self.welcome_label.config(text=result_parts[1])
        else:
            self.welcome_label.config(text=result)
        self.displayed_widgets["higher_button"].config(state=tk.DISABLED)
        self.displayed_widgets["lower_button"].config(state=tk.DISABLED)

    def guess_lower(self):
        result = self.games.guess_card(higher=False)
        self.displayed_widgets["card_result_label"].config(text=result)
        result_parts = result.split("\n")
        if len(result_parts) > 1:
            self.welcome_label.config(text=result_parts[1])
        else:
            self.welcome_label.config(text=result)
        self.displayed_widgets["higher_button"].config(state=tk.DISABLED)
        self.displayed_widgets["lower_button"].config(state=tk.DISABLED)

    # --- Dice Roll Target Sum Game ---
    def play_dice_roll(self):
        num_dice = self.displayed_widgets["num_dice_entry"].get()
        target_sum = self.displayed_widgets["target_sum_entry"].get()
        result = self.games.play_dice_roll(num_dice, target_sum)
        result_parts = result.split("\n")

        if len(result_parts) > 1:
            self.displayed_widgets["dice_result_label"].config(text=result_parts[0])
            self.welcome_label.config(text=result_parts[1])
        else:
            self.welcome_label.config(text=result)
            self.displayed_widgets["dice_result_label"].config(text="")


# Implementation for Google Colab (ipywidgets)
class ProbabilityGamesColab:
    def __init__(self):
        self.games = ProbabilityGames()
        self.output = widgets.Output()

        # Game selection dropdown
        self.game_dropdown = widgets.Dropdown(
            options=['Coin Toss Guessing Game', 'Card Rank Higher/Lower', 'Dice Roll Target Sum'],
            value='Coin Toss Guessing Game',
            description='Select Game:',
            disabled=False,
        )
        self.game_dropdown.observe(self.on_game_change, names='value')

        # Welcome label
        self.welcome_label = widgets.HTML(value="<h3>Welcome to Interesting Probability Games!</h3>")

        # Container for game-specific widgets
        self.game_container = widgets.VBox()

        # Main layout
        self.layout = widgets.VBox([
            self.game_dropdown,
            self.welcome_label,
            self.game_container,
            self.output
        ])

        # Initialize with first game
        self.show_game_widgets(self.game_dropdown.value)

    def on_game_change(self, change):
        with self.output:
            clear_output()
        self.welcome_label.value = f"<h3>Welcome to {change.new}!</h3>"
        self.show_game_widgets(change.new)

    def show_game_widgets(self, game):
        game_widgets = []

        if game == "Coin Toss Guessing Game":
            self.coin_guess_entry = widgets.Text(description="Guess:", placeholder="Heads or Tails")
            self.flip_button = widgets.Button(description="Flip Coin")
            self.flip_button.on_click(self.play_coin_toss)
            game_widgets = [self.coin_guess_entry, self.flip_button]

        elif game == "Card Rank Higher/Lower":
            self.card_drawn_label = widgets.HTML(value="<p>Click 'Draw' to start.</p>")
            self.draw_button = widgets.Button(description="Draw Card")
            self.draw_button.on_click(self.draw_new_card)
            self.higher_button = widgets.Button(description="Higher", disabled=True)
            self.higher_button.on_click(self.guess_higher)
            self.lower_button = widgets.Button(description="Lower", disabled=True)
            self.lower_button.on_click(self.guess_lower)
            game_widgets = [self.card_drawn_label, self.draw_button,
                           widgets.HBox([self.higher_button, self.lower_button])]

        elif game == "Dice Roll Target Sum":
            self.num_dice_entry = widgets.IntText(value=2, description="Number of Dice:", min=1)
            self.target_sum_entry = widgets.IntText(description="Target Sum:")
            self.roll_dice_button = widgets.Button(description="Roll Dice")
            self.roll_dice_button.on_click(self.play_dice_roll)
            game_widgets = [self.num_dice_entry, self.target_sum_entry, self.roll_dice_button]

        self.game_container.children = game_widgets

    def play_coin_toss(self, b):
        with self.output:
            clear_output()
            result = self.games.play_coin_toss(self.coin_guess_entry.value)
            print(result)
            self.welcome_label.value = f"<h3>{result}</h3>"

    def draw_new_card(self, b):
        with self.output:
            clear_output()
            card_text = self.games.draw_new_card()
            print(card_text)
            self.card_drawn_label.value = f"<p>{card_text}</p>"
            self.welcome_label.value = "<h3>Card drawn! Will the next card be higher or lower?</h3>"
            self.higher_button.disabled = False
            self.lower_button.disabled = False

    def guess_higher(self, b):
        with self.output:
            clear_output()
            result = self.games.guess_card(higher=True)
            print(result)
            # Split the result and get the last part (avoiding any backslash issues)
            parts = result.split('\n')
            if len(parts) > 1:
                self.welcome_label.value = f"<h3>{parts[-1]}</h3>"
            else:
                self.welcome_label.value = f"<h3>{result}</h3>"
            self.higher_button.disabled = True
            self.lower_button.disabled = True

    def guess_lower(self, b):
        with self.output:
            clear_output()
            result = self.games.guess_card(higher=False)
            print(result)
            # Split the result and get the last part (avoiding any backslash issues)
            parts = result.split('\n')
            if len(parts) > 1:
                self.welcome_label.value = f"<h3>{parts[-1]}</h3>"
            else:
                self.welcome_label.value = f"<h3>{result}</h3>"
            self.higher_button.disabled = True
            self.lower_button.disabled = True

    def play_dice_roll(self, b):
        with self.output:
            clear_output()
            result = self.games.play_dice_roll(self.num_dice_entry.value, self.target_sum_entry.value)
            print(result)
            # Split the result and get the parts (avoiding any backslash issues)
            parts = result.split('\n')
            if len(parts) > 1:
                self.welcome_label.value = f"<h3>{parts[1]}</h3>"
            else:
                self.welcome_label.value = f"<h3>{result}</h3>"


# Main execution
if __name__ == "__main__":
    if IN_COLAB:
        # Google Colab implementation
        colab_app = ProbabilityGamesColab()
        display(colab_app.layout)
    else:
        # VS Code / regular Python implementation
        root = tk.Tk()
        gui = ProbabilityGamesGUI(root)
        root.mainloop()